In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

#Load Model (Trained)

In [2]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

base_model = SentenceTransformer("indobenchmark/indobert-base-p1")
model = SentenceTransformer("ShoAnn/retriever_checkpoints")

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/709k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

#Load and Prepare Test Dataset

In [3]:
#@title Compute embedding
from datasets import load_dataset
import pandas as pd
import torch

qpn_dataset = load_dataset('ShoAnn/indo_legalqa_triplets')

lb_all = pd.read_csv(
    "/content/drive/MyDrive/SKRIPSI/lb_v1_5474.csv",
    index_col=0,
    dtype={
        "pasal":"Int64",
        "angka":"Int64",
        "nomor":"Int64",
        "tahun":"Int64",
    })
queries = list(set(qpn_dataset["test"]["q"]))
corpus = list(lb_all['fulltext'])

corpus_embeddings_base = base_model.encode(corpus, convert_to_tensor=True)
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

README.md:   0%|          | 0.00/629 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/319k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4876 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/610 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/610 [00:00<?, ? examples/s]

In [4]:
#@title Prepare ground truth
ground_truth = pd.DataFrame(set(qpn_dataset["test"]["q"]), columns=["query"])
def group_pos(query):
    pos_list = []
    for i in range(len(qpn_dataset["test"]["q"])):
        if qpn_dataset["test"]["q"][i] == query:
            pos_list.append(qpn_dataset["test"]["pos"][i])
    return pos_list
ground_truth["docs"] = ground_truth["query"].apply(group_pos)
ground_truth

,query,docs
0,"Belakangan ini, beredar berita tentang kasus b...",[Undang-Undang ini mulai berlaku setelah 3 (ti...
1,Bolehkah melakukan aborsi terhadap kehamilan a...,[Disarikan dari Aborsi dalam Perspektif Hukum ...
2,Apakah penjara anak dan orang dewasa menjadi s...,[['(1) sistem peradilan pidana anak wajib men...
3,Bagaimana tata cara persidangan perkara pidana...,[(2) Dalam hal sidang dilaksanakan secara elek...
4,Bagaimana sikap hakim seharusnya jika menemuka...,[['(1) dalam hal tersangka atau terdakwa disa...
...,...,...
443,Adakah sanksi hukum nonton film porno bagi ora...,"[pasal 12 setiap orang dilarang mengajak, me..."
444,Bagaimanakah penerapan sanksi pidana terhadap ...,"[Namun, pada intinya dapat disimpulkan bahwa k..."
445,Saya merekam (kejadian tanpa sengaja) pembicar...,[['(1) atau ayat']]
446,Apakah laporan dapat dicabut kembali? Dalam ha...,[Tiap jumlah maksimum hukuman denda yang dianc...


In [5]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
from tqdm import tqdm

def retrieve_docs(queries, top_k, model, corpus_embeddings):
    retrieved_docs = []
    for query in queries:
        query_embedding = model.encode(query, convert_to_tensor=True)

        similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
        scores, indices = torch.topk(similarity_scores, k=top_k)

        docs = []
        for score, idx in zip(scores, indices):
            docs.append((corpus[idx], score.detach().cpu().item()))
        retrieved_docs.append({"query": query, "docs": docs})
    return retrieved_docs

In [6]:
k5_base = pd.DataFrame(retrieve_docs(queries, 5, base_model, corpus_embeddings_base))
k10_base = pd.DataFrame(retrieve_docs(queries, 10, base_model, corpus_embeddings_base))
k20_base = pd.DataFrame(retrieve_docs(queries, 20, base_model, corpus_embeddings_base))
k100_base = pd.DataFrame(retrieve_docs(queries, 100, base_model, corpus_embeddings_base))

k5 = pd.DataFrame(retrieve_docs(queries, 5, model, corpus_embeddings))
k10 = pd.DataFrame(retrieve_docs(queries, 10, model, corpus_embeddings))
k20 = pd.DataFrame(retrieve_docs(queries, 20, model, corpus_embeddings))
k100 = pd.DataFrame(retrieve_docs(queries, 100, model, corpus_embeddings))

#Compute Metrics

In [7]:
class TestSet:
    def __init__(self, ground_truth_df, test_df):
        self.ground_truth_df = ground_truth_df
        self.test_df = test_df
    def __recall(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        tp = [doc for doc in test_docs if doc in gt_list]
        recall = len(tp) / len(gt_list) # TP / (TP + FN) where FN = len(gt_list) - len(tp)
        return recall

    def get_recall_score(self, k):
        recall = 0
        for _, row in self.test_df.iterrows():
            recall += self.__recall(row)
        return recall / len(self.test_df)

    def __reciprocal_rank(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        for rank, doc_id in enumerate(test_docs, 1):
            if doc_id in gt_list:
                return 1.0 / rank
        return 0.0

    def get_mrr_score(self, k):
        rr_sum = 0
        for _, row in self.test_df.iterrows():
            rr_sum += self.__reciprocal_rank(row)
        return rr_sum / len(self.test_df) if len(self.test_df) > 0 else 0.0


## base model

In [9]:
test_k_5_base = TestSet(ground_truth, k5_base)
test_k_10_base = TestSet(ground_truth, k10_base)
test_k_20_base = TestSet(ground_truth, k20_base)
test_k_100_base = TestSet(ground_truth, k100_base)

recall_k_5_base = test_k_5_base.get_recall_score(5)
recall_k_10_base = test_k_10_base.get_recall_score(10)
recall_k_20_base = test_k_20_base.get_recall_score(20)
recall_k_100_base = test_k_100_base.get_recall_score(100)

print(f"Recall@5: {recall_k_5_base}")
print(f"Recall@10: {recall_k_10_base}")
print(f"Recall@20: {recall_k_20_base}")
print(f"Recall@100: {recall_k_100_base}")

mrr_k_5_base = test_k_5_base.get_mrr_score(5)
mrr_k_10_base = test_k_10_base.get_mrr_score(10)
mrr_k_20_base = test_k_20_base.get_mrr_score(20)
mrr_k_100_base = test_k_100_base.get_mrr_score(100)

print(f"mrr@5: {mrr_k_5_base}")
print(f"mrr@10: {mrr_k_10_base}")
print(f"mrr@20: {mrr_k_20_base}")
print(f"mrr@100: {mrr_k_100_base}")

Recall@5: 0.028050595238095232
Recall@10: 0.05632440476190476
Recall@20: 0.09441964285714285
Recall@100: 0.29769345238095235
mrr@5: 0.025669642857142856
mrr@10: 0.02953071853741496
mrr@20: 0.03110301649068718
mrr@100: 0.03322089419132728


## fine tuned model

In [12]:
test_k_5 = TestSet(ground_truth, k5)
test_k_10 = TestSet(ground_truth, k10)
test_k_20 = TestSet(ground_truth, k20)
test_k_100 = TestSet(ground_truth, k100)

recall_k_5 = test_k_5.get_recall_score(5)
recall_k_10 = test_k_10.get_recall_score(10)
recall_k_20 = test_k_20.get_recall_score(20)
recall_k_100 = test_k_100.get_recall_score(100)

print(f"Recall@5: {recall_k_5}")
print(f"Recall@10: {recall_k_10}")
print(f"Recall@20: {recall_k_20}")
print(f"Recall@100: {recall_k_100}")

mrr_k_5 = test_k_5.get_mrr_score(5)
mrr_k_10 = test_k_10.get_mrr_score(10)
mrr_k_20 = test_k_20.get_mrr_score(20)
mrr_k_100 = test_k_100.get_mrr_score(100)

print(f"mrr@5: {mrr_k_5}")
print(f"mrr@10: {mrr_k_10}")
print(f"mrr@20: {mrr_k_20}")
print(f"mrr@100: {mrr_k_100}")

Recall@5: 0.0
Recall@10: 0.005208333333333334
Recall@20: 0.005952380952380952
Recall@100: 0.14516369047619052
mrr@5: 0.0
mrr@10: 0.000496031746031746
mrr@20: 0.000698953823953824
mrr@100: 0.0010622188620138542
